In [289]:
import pandas as pd
import numpy as np
from random import randint

In [243]:
# Import tabula to read pdf file
import tabula

In [244]:
mak23 = tabula.read_pdf("examples/Mak2023-2024.pdf", pages="all")

D:\Novel\Data\Python\DataAnalysis\DataAnalysisVenv\Lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [532]:
# This data is rough and improperly structured
# length of mak23
len(mak23)

335

In [541]:
columnNames = ["INDEX NO", "NAME", "GENDER", "UACE YEAR", "CODE", "DISTRICT", "COURSE CODE", "COURSE NAME"]

In [535]:
def instTest(x):
    try: 
        if isinstance(float(x), float):
            return True
    except ValueError:
        return False

In [606]:
# Function to extract data from data2
def dataProcess(n):
#     if n >= 1:
    studentData = data[n].tolist()
    index_no = studentData[0]
    name = " ".join(studentData[1:-7])[:-1]
    gender = studentData[-8][-1]
    UACE_YEAR = int(studentData[-7])
    DistrictCode = int(studentData[-6])
    District = studentData[-5]
    CourseCode = studentData[-4]
    Course = ' '.join(studentData[-3:])
    return [index_no, name, gender, UACE_YEAR,
            DistrictCode, District, CourseCode, Course]

def getStudentData(data):
    studentList = []
    for i in range(len(data)):
    #     if i > 0:
        student = dataProcess(i)
        studentList.append(student)
    return studentList

In [585]:
def getDf():
    # i = randint(0, len(mak23)-1)
    # i = 0
    # print(i)

    # # First column is a common to all, drop it
    # # df = mak23[i]
    # columns = df.columns
    # print("columns: ", columns)
    # df = df.drop(columns[0], axis=1)

    # # Drop columns with nan values
    # print("columns before dropping nan-columns: ", df.shape[1])
    # df = df.dropna(axis=1,
    #                # thresh=3,
    #                how="all",
    #               )
    # print("columns after dropping: ", df.shape[1])

    # # drop rows with nan if they exist
    # print("Rows before dropping: ", df.shape[0])
    # df = df.dropna(axis=0,
    #                thresh=3,
    #               )
    
    # if len(df.columns) == 3:
    #     # drop all columns with rows with nan
    #     print("Rows after dropping: ", df.shape[0])
        
    #     df = df.dropna(axis=0, thresh=3)
    #     # Drop rows for which Tot_wt is not float
    #     df = df[df[columns[-1]] != "Tot_wt"]

    #     # empty data frame
    #     # df_temp = pd.DataFrame(data=[],
    #     #                        columns=columnNames
    #     #                       )
    # # data = df[columns[1]]
    #     # print(data)
    df_temp = getStudentData(data)
    return df_temp
        # return data
    
    # return df

In [605]:
data.tolist()[0]

'U0391/673 LUKABWE DOUGLAS MARKM 2022 42 MUKONO ACC BSC. Accounting (EVE)'

In [607]:
studentList = []
for i in range(len(data)):
    #     if i > 0:
    student = dataProcess(i)
    studentList.append(student)

KeyError: 0

In [581]:
data = data1['MAKERERE UIVERSITY']

In [583]:
data

5     U0391/673 LUKABWE DOUGLAS MARKM 2022 42 MUKONO...
6     U1249/532 LULE ISAACM 2022 40 MPIGI ACC BSC. A...
7     U0459/676 HAULAH SULAIT BIIRAF 2022 21 KASESE ...
8     U1611/887 NAMBALYE EVELYNEF 2022 33 MASAKA ACC...
9     U0083/555 KEMIGISHA  RUSIAF 2022 65 IBANDA ACC...
10    U2373/501 LUWAGA TONNYM 2018 55 WAKISO ACC BSC...
11    U0856/511 NAMAGEMBE MARTINAF 2022 25 KIBOGA AC...
12    U2929/521 KALUNGI TITUSM 2022 75 MITYANA ACC B...
13    U1609/550 KIZITO PATRICKM 2011 16 KAMPALA ACC ...
14    U1224/882 NDYOMUHIRWA CHRISTIANM 2022 50 RUKUN...
15    U1350/503 GWOKYALYA RESTYF 2022 16 KAMPALA ACC...
16    U0086/599 WAISWA CEASERM 2009 17 KAMULI ACC BS...
17    U0923/523 NABUUMA MAJOREENF 2017 55 WAKISO ACC...
18    U2789/821 TAMALE COLINM 2022 91 GOMBA ACC BSC....
19    U0960/506 LUUTU DOUGLASM 2022 16 KAMPALA ACC B...
20    U0459/684 KASANGAKI MARIAM BINTAF 2022 13 KABA...
21    U1611/947 NANSUBUGA  LYDIAF 2022 16 KAMPALA AC...
22    U3219/507 NANDAGIRE SCOVIAF 2022 49 RAKAI 

In [584]:
getDf()

0
columns:  Index(['Unnamed: 0', 'Unnamed: 1', 'MAKERERE UIVERSITY', 'Unnamed: 2'], dtype='object')
columns before dropping nan-columns:  3
columns after dropping:  3
Rows before dropping:  45
Rows after dropping:  41


KeyError: 0

In [246]:
# Index the first table in the mak23 pdf
# Data is highly unordered and requires pre-processing
# A number of techniques shall be used to clean and prepare data into proper structured
# tabular format.
mak23[0]

,Unnamed: 0,Unnamed: 1,MAKERERE UIVERSITY,Unnamed: 2
0,NaN,NaN,ACADEMIC REGISTRAR'S DEPARTMENT,NaN
1,NaN,NaN,ADMITTED APPLICANTS FOR PRIVATE SPONSORSHIP FO...,NaN
2,NaN,NaN,BSC. Accounting (EVE),NaN
3,No,formid,Index_no name gender uace_yr code District cou...,Tot_wt
4,NaN,NaN,code,NaN
5,1,23APF412304454141015,U0391/673 LUKABWE DOUGLAS MARKM 2022 42 MUKONO...,42.2
6,2,23APF345674424112360,U1249/532 LULE ISAACM 2022 40 MPIGI ACC BSC. A...,41.1
7,3,23APF1361730063122187,U0459/676 HAULAH SULAIT BIIRAF 2022 21 KASESE ...,40.6
8,4,23APF519647933132646,U1611/887 NAMBALYE EVELYNEF 2022 33 MASAKA ACC...,38.8
9,5,23APF1481620160129238,U0083/555 KEMIGISHA RUSIAF 2022 65 IBANDA ACC...,38.4


In [247]:
# Drop missing data rows.
# It can be observed that index 3 contains what would have been column names: Form id, Index_no, ...
# It also contains original data index read as column: Unnamed: 0
data1 = mak23[0].dropna()

In [248]:
data1

,Unnamed: 0,Unnamed: 1,MAKERERE UIVERSITY,Unnamed: 2
3,No,formid,Index_no name gender uace_yr code District cou...,Tot_wt
5,1,23APF412304454141015,U0391/673 LUKABWE DOUGLAS MARKM 2022 42 MUKONO...,42.2
6,2,23APF345674424112360,U1249/532 LULE ISAACM 2022 40 MPIGI ACC BSC. A...,41.1
7,3,23APF1361730063122187,U0459/676 HAULAH SULAIT BIIRAF 2022 21 KASESE ...,40.6
8,4,23APF519647933132646,U1611/887 NAMBALYE EVELYNEF 2022 33 MASAKA ACC...,38.8
9,5,23APF1481620160129238,U0083/555 KEMIGISHA RUSIAF 2022 65 IBANDA ACC...,38.4
10,6,23APF1501202078128326,U2373/501 LUWAGA TONNYM 2018 55 WAKISO ACC BSC...,38.4
11,7,23APF1139056282132364,U0856/511 NAMAGEMBE MARTINAF 2022 25 KIBOGA AC...,38.2
12,8,23APF101396019107326,U2929/521 KALUNGI TITUSM 2022 75 MITYANA ACC B...,38.1
13,9,23APF515017899120808,U1609/550 KIZITO PATRICKM 2011 16 KAMPALA ACC ...,37.9


In [249]:
# This returns the columns in the DataFrame
data1.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'MAKERERE UIVERSITY', 'Unnamed: 2'], dtype='object')

In [250]:
# drop column: Unnamed: 0
del data1["Unnamed: 0"]

In [251]:
len(data1['MAKERERE UIVERSITY'])

41

In [252]:
# Key data is in column Makerere University
# Employ python string processing methods to extracts such data

In [253]:
data1.iloc[0]

Unnamed: 1                                                       formid
MAKERERE UIVERSITY    Index_no name gender uace_yr code District cou...
Unnamed: 2                                                       Tot_wt
Name: 3, dtype: object

In [254]:
data1 = data1.drop(index=3)

In [255]:
# data1.index

In [562]:
# Reindex the dataframe
data2 = data1.reindex([x for x in range(len(data1.index))])

In [563]:
data2

,Unnamed: 1,MAKERERE UIVERSITY,Unnamed: 2
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,23APF412304454141015,U0391/673 LUKABWE DOUGLAS MARKM 2022 42 MUKONO...,42.2
6,23APF345674424112360,U1249/532 LULE ISAACM 2022 40 MPIGI ACC BSC. A...,41.1
7,23APF1361730063122187,U0459/676 HAULAH SULAIT BIIRAF 2022 21 KASESE ...,40.6
8,23APF519647933132646,U1611/887 NAMBALYE EVELYNEF 2022 33 MASAKA ACC...,38.8
9,23APF1481620160129238,U0083/555 KEMIGISHA RUSIAF 2022 65 IBANDA ACC...,38.4


In [258]:
newIndex = [x for x in range(len(data1.index))]
type(newIndex)

list

In [579]:
getStudentData(data2)

IndexError: list index out of range

In [260]:
# data[1]
data = [x for x in data2[1:]]
# data

In [551]:
# Preprocess and extract migled data from 'MAKERERE UIVERSITY' column
data2 = data1['MAKERERE UIVERSITY']

In [552]:
data2

5     U0391/673 LUKABWE DOUGLAS MARKM 2022 42 MUKONO...
6     U1249/532 LULE ISAACM 2022 40 MPIGI ACC BSC. A...
7     U0459/676 HAULAH SULAIT BIIRAF 2022 21 KASESE ...
8     U1611/887 NAMBALYE EVELYNEF 2022 33 MASAKA ACC...
9     U0083/555 KEMIGISHA  RUSIAF 2022 65 IBANDA ACC...
10    U2373/501 LUWAGA TONNYM 2018 55 WAKISO ACC BSC...
11    U0856/511 NAMAGEMBE MARTINAF 2022 25 KIBOGA AC...
12    U2929/521 KALUNGI TITUSM 2022 75 MITYANA ACC B...
13    U1609/550 KIZITO PATRICKM 2011 16 KAMPALA ACC ...
14    U1224/882 NDYOMUHIRWA CHRISTIANM 2022 50 RUKUN...
15    U1350/503 GWOKYALYA RESTYF 2022 16 KAMPALA ACC...
16    U0086/599 WAISWA CEASERM 2009 17 KAMULI ACC BS...
17    U0923/523 NABUUMA MAJOREENF 2017 55 WAKISO ACC...
18    U2789/821 TAMALE COLINM 2022 91 GOMBA ACC BSC....
19    U0960/506 LUUTU DOUGLASM 2022 16 KAMPALA ACC B...
20    U0459/684 KASANGAKI MARIAM BINTAF 2022 13 KABA...
21    U1611/947 NANSUBUGA  LYDIAF 2022 16 KAMPALA AC...
22    U3219/507 NANDAGIRE SCOVIAF 2022 49 RAKAI 

In [553]:
# Function to extract data from data2
def dataProcess(n):
#     if n >= 1:
    studentData = data[n].split()
    index_no = studentData[0]
    name = " ".join(studentData[1:-7])[:-1]
    gender = studentData[-8][-1]
    UACE_YEAR = int(studentData[-7])
    DistrictCode = int(studentData[-6])
    District = studentData[-5]
    CourseCode = studentData[-4]
    Course = ' '.join(studentData[-3:])
    return [index_no, name, gender, UACE_YEAR,
            DistrictCode, District, CourseCode, Course]

In [264]:
# Create a data frame with no data
df = pd.DataFrame([], columns=DataIndex)

In [265]:
df

,Index No,NAME,GENDER,UACE YEAR,CODE,DISTRICT,COURSE CODE,COURSE NAME


In [266]:
# data

In [554]:
studentList = []
for i in range(len(data)):
#     if i > 0:
    student = dataProcess(i)
    studentList.append(student)

In [555]:
studentList

[['U1249/532',
  'LULE ISAAC',
  'M',
  2022,
  40,
  'MPIGI',
  'ACC',
  'BSC. Accounting (EVE)'],
 ['U0459/676',
  'HAULAH SULAIT BIIRA',
  'F',
  2022,
  21,
  'KASESE',
  'ACC',
  'BSC. Accounting (EVE)'],
 ['U1611/887',
  'NAMBALYE EVELYNE',
  'F',
  2022,
  33,
  'MASAKA',
  'ACC',
  'BSC. Accounting (EVE)'],
 ['U0083/555',
  'KEMIGISHA RUSIA',
  'F',
  2022,
  65,
  'IBANDA',
  'ACC',
  'BSC. Accounting (EVE)'],
 ['U2373/501',
  'LUWAGA TONNY',
  'M',
  2018,
  55,
  'WAKISO',
  'ACC',
  'BSC. Accounting (EVE)'],
 ['U0856/511',
  'NAMAGEMBE MARTINA',
  'F',
  2022,
  25,
  'KIBOGA',
  'ACC',
  'BSC. Accounting (EVE)'],
 ['U2929/521',
  'KALUNGI TITUS',
  'M',
  2022,
  75,
  'MITYANA',
  'ACC',
  'BSC. Accounting (EVE)'],
 ['U1609/550',
  'KIZITO PATRICK',
  'M',
  2011,
  16,
  'KAMPALA',
  'ACC',
  'BSC. Accounting (EVE)'],
 ['U1224/882',
  'NDYOMUHIRWA CHRISTIAN',
  'M',
  2022,
  50,
  'RUKUNGIRI',
  'ACC',
  'BSC. Accounting (EVE)'],
 ['U1350/503',
  'GWOKYALYA RESTY',
  'F

In [556]:
df = pd.DataFrame(studentList, columns=DataIndex)

In [557]:
df

,Index No,NAME,GENDER,UACE YEAR,CODE,DISTRICT,COURSE CODE,COURSE NAME
0,U1249/532,LULE ISAAC,M,2022,40,MPIGI,ACC,BSC. Accounting (EVE)
1,U0459/676,HAULAH SULAIT BIIRA,F,2022,21,KASESE,ACC,BSC. Accounting (EVE)
2,U1611/887,NAMBALYE EVELYNE,F,2022,33,MASAKA,ACC,BSC. Accounting (EVE)
3,U0083/555,KEMIGISHA RUSIA,F,2022,65,IBANDA,ACC,BSC. Accounting (EVE)
4,U2373/501,LUWAGA TONNY,M,2018,55,WAKISO,ACC,BSC. Accounting (EVE)
5,U0856/511,NAMAGEMBE MARTINA,F,2022,25,KIBOGA,ACC,BSC. Accounting (EVE)
6,U2929/521,KALUNGI TITUS,M,2022,75,MITYANA,ACC,BSC. Accounting (EVE)
7,U1609/550,KIZITO PATRICK,M,2011,16,KAMPALA,ACC,BSC. Accounting (EVE)
8,U1224/882,NDYOMUHIRWA CHRISTIAN,M,2022,50,RUKUNGIRI,ACC,BSC. Accounting (EVE)
9,U1350/503,GWOKYALYA RESTY,F,2022,16,KAMPALA,ACC,BSC. Accounting (EVE)


In [271]:
# Insert Total Weight Column into the DataFrame
df.insert(8, "Total Weight", pd.Series([x for x in weights], index=df.index))

In [272]:
# Insert formId Column as the first column
# Qn: Can we insert multiple columns at once🤔
df.insert(0, "Form ID", pd.Series([x for x in formId], index=df.index))

In [273]:
df

,Form ID,Index No,NAME,GENDER,UACE YEAR,CODE,DISTRICT,COURSE CODE,COURSE NAME,Total Weight
0,23APF345674424112360,U1249/532,LULE ISAAC,M,2022,40,MPIGI,ACC,BSC. Accounting (EVE),41.1
1,23APF1361730063122187,U0459/676,HAULAH SULAIT BIIRA,F,2022,21,KASESE,ACC,BSC. Accounting (EVE),40.6
2,23APF519647933132646,U1611/887,NAMBALYE EVELYNE,F,2022,33,MASAKA,ACC,BSC. Accounting (EVE),38.8
3,23APF1481620160129238,U0083/555,KEMIGISHA RUSIA,F,2022,65,IBANDA,ACC,BSC. Accounting (EVE),38.4
4,23APF1501202078128326,U2373/501,LUWAGA TONNY,M,2018,55,WAKISO,ACC,BSC. Accounting (EVE),38.4
5,23APF1139056282132364,U0856/511,NAMAGEMBE MARTINA,F,2022,25,KIBOGA,ACC,BSC. Accounting (EVE),38.2
6,23APF101396019107326,U2929/521,KALUNGI TITUS,M,2022,75,MITYANA,ACC,BSC. Accounting (EVE),38.1
7,23APF515017899120808,U1609/550,KIZITO PATRICK,M,2011,16,KAMPALA,ACC,BSC. Accounting (EVE),37.9
8,23APF1672224247102423,U1224/882,NDYOMUHIRWA CHRISTIAN,M,2022,50,RUKUNGIRI,ACC,BSC. Accounting (EVE),37.9
9,23APF202185286140690,U1350/503,GWOKYALYA RESTY,F,2022,16,KAMPALA,ACC,BSC. Accounting (EVE),37.8


<h3>Assignment: Preprocess the remaining data tables to have them similar to the data above.</h3>

In [274]:
# Are there better ways of doing the above tasks?

# <h3>Data description</h3>

In [275]:
# which district had a higher intake and what is its frequency
df.DISTRICT.describe()

count         39
unique        25
top       WAKISO
freq           5
Name: DISTRICT, dtype: object

In [276]:
# In the data portion, there are 26 unique districts.
# Wakiso has oneof the highest frequency of student admissions.
# Budaka is one of the districts with the lowest frequency admissions.

In [277]:
# why is Budaka returned as the district with the lowest frequency
df.DISTRICT.min()

'BUDAKA'

In [278]:
# why is Wakiso returned as the district with the highest frequency
df.DISTRICT.max()

'WAKISO'

In [279]:
df.DISTRICT.value_counts()

DISTRICT
WAKISO        5
KAMPALA       5
HOIMA         3
MASAKA        2
BUTAMBALA     2
KAMWENGE      2
JINJA         2
MPIGI         1
LWENGO        1
MUBENDE       1
BUDAKA        1
BUNYANGABU    1
KYENJOJO      1
KAGADI        1
RAKAI         1
IGANGA        1
KASESE        1
KABAROLE      1
GOMBA         1
KAMULI        1
RUKUNGIRI     1
MITYANA       1
KIBOGA        1
IBANDA        1
BUYENDE       1
Name: count, dtype: int64

In [280]:
# analysis by SEX

In [281]:
# More males than females were admitted
df.GENDER.value_counts()

GENDER
M    22
F    17
Name: count, dtype: int64

In [282]:
# Count gender by district 
df_district_gender = pd.DataFrame(data = df.GENDER.value_counts(), index = list(df.DISTRICT), columns = list(df.GENDER))

In [283]:
# How can we get district gender count?